In [1]:
from langchain.llms.llamacpp import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from concurrent.futures import ProcessPoolExecutor
import asyncio

In [2]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [4]:
def create_model():
    return LlamaCpp(
        model_path="./gpt4all-falcon-q4_0.gguf",
        temperature=0.75,
        # max_tokens=2000,
        top_p=1,
        callback_manager=callback_manager,
        verbose=True,  # Verbose is required to pass to the callback manager
        streaming=False,
    ) # type: ignore

In [5]:
pool = ProcessPoolExecutor(max_workers=4, initializer=create_model)

In [6]:
def model_predict(question: str):
    prompt = f"Question: {question}"
    llm = create_model()
    result = llm(prompt)
    return result

In [7]:
loop = asyncio.get_event_loop()

answer = await loop.run_in_executor(pool, model_predict(question="Hello"))

answer

ImportError: Could not import llama-cpp-python library. Please install the llama-cpp-python library to use this embedding model: pip install llama-cpp-python